In [1]:
# Importing libraries

'''
To import any module, run following command in ipynb
import sys
!{sys.executable} -m pip install wordcloud
'''

import matplotlib.pyplot as plt
from pandas import DataFrame


from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
import pickle
import gensim

from ipynb.fs.full.Data_preprocessing import clean

import xgboost as xgb

In [2]:
# Function to find cosine similarity between two vectors tfidf_1, tfidf_2

def cosine_similarity(tfidf_1, tfidf_2):
    return cosine(tfidf_1.todense(),tfidf_2.todense())

In [3]:
df = pd.read_csv('AnsData.csv', header = None, sep='\t' )

In [4]:
df.head()

,0,1,2,3,4
0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,1,I am going to ignore your question and share 5...,I am going to ignore your question and share 5...
1,what is the story of kohinoor kohinoor diamond,what would happen if the indian government sto...,0,History of Kohinoor DiamondThe Kohinoor is one...,"Forget about attacking, No one on earth can at..."
2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,Hey! There are few options to increase the spe...,"Using DNS functionality, you can use your comp..."
3,why am i mentally very lonely how can i solve it,find the remainder when math2324math is divide...,0,While reading your question I pictured myself ...,Using the J programming language:a#~ *./23 35=...
4,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water,0,"Not enough data to answer, you need to add the...","Ah, the old “drown ’em or desiccate ‘em” quest..."


In [5]:

model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
model.init_sims(replace=True)

In [6]:

def wmd(s1,s2):
    return model.wmdistance(s1, s2)

In [7]:

def distance(s1, s2):
    
    
    num = len(s1)
          
    sent1 = [word for word in s1.split() if word in model.vocab]
    sent2 = [word for word in s2.split() if word in model.vocab]
        
    if len(sent1)>0 and len(sent2)>0:
        return model.wmdistance(sent1,sent2)
    else:
        return 1000
        
       
        

In [8]:
data = pd.read_csv("quora_duplicate_questions.tsv", sep="\t")


from ipynb.fs.full.Data_preprocessing import clean

data['question1'] = data['question1'].apply(clean)
data['question2'] = data['question2'].apply(clean)



In [9]:

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(max_features = 50000)
tfidf_vect.fit(pd.Series(data['question1']+data['question2']) .astype(str).unique() ) 

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=50000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [10]:
dist_ans = []
dist_que = []
tdist_ans = []
tdist_que = []

for i in range(931):
    da = distance(df[3][i], df[4][i])
    dist_ans.append(da)
    dq = distance(df[0][i], df[1][i])
    dist_que.append(dq)
    
    t1 = tfidf_vect.transform([df[3][i]])
    t2 = tfidf_vect.transform([df[4][i]])
    tda = cosine_similarity(t1, t2)
    tdist_ans.append(tda)
    
    t1 = tfidf_vect.transform([df[0][i]])
    t2 = tfidf_vect.transform([df[1][i]])
    tdq = cosine_similarity(t1, t2)
    tdist_que.append(tdq)
    
    
    
    
    print("{:<15d}{:<15f}{:<15f}{:<15f}{:<15f}".format(i, da, dq, tda, tdq ))

0              0.000000       0.180997       0.000000       0.020760       
1              0.939044       0.790473       0.803154       0.200320       
2              0.929330       0.756303       0.813780       0.745459       
3              1.086222       1.220441       0.926564       1.000000       
4              1.015583       0.801635       0.912896       0.741938       
5              0.000000       0.463670       0.000000       0.458869       
6              0.979779       1.290149       0.849560       1.000000       
7              0.000000       0.492047       0.000000       0.217224       
8              1.114567       0.000000       0.975416       0.103848       
9              0.000000       0.773614       0.000000       0.273015       
10             0.977883       1.270182       0.961101       0.982118       
11             0.000000       0.279703       0.000000       0.048593       
12             0.000000       0.183809       0.000000       0.046952       
13          

108            0.779182       0.431437       0.720909       0.261458       
109            0.885329       1.215930       0.858298       1.000000       
110            0.000000       0.970555       0.000000       0.447608       
111            0.000000       0.548239       0.000000       0.282848       
112            1.033415       0.806524       0.893183       0.579624       
113            0.821847       0.169528       0.788933       0.478422       
114            0.928526       0.760784       0.897647       0.987081       
115            0.993957       0.604571       0.722145       0.280909       
116            0.827332       0.804785       0.762698       0.338997       
117            1.190191       0.680398       0.961789       0.552969       
118            0.996478       0.587571       0.829110       0.488643       
119            0.789552       0.887271       0.688985       0.486046       
120            1.198899       0.549210       0.945327       0.199243       
121         

217            0.802136       0.676175       0.643616       0.310460       
218            0.852995       0.761276       0.803397       0.680777       
219            0.000000       0.548742       0.000000       0.537843       
220            0.840519       0.696452       0.717675       0.461795       
221            0.000000       0.545906       0.000000       0.425499       
222            0.886746       0.671172       0.836753       0.643875       
223            0.684500       0.192979       0.565843       0.042226       
224            0.882757       0.305682       0.664309       0.569448       
225            0.917804       0.828967       0.819947       0.478790       
226            0.881997       0.828239       0.868114       0.598431       
227            1.132680       0.465372       0.951278       0.486300       
228            0.826659       0.991484       0.717340       0.864059       
229            0.768302       1.031825       0.592578       0.832708       
230         

325            0.853977       0.572116       0.702356       0.648083       
326            0.000000       0.131990       0.000000       0.027445       
327            0.851789       0.936029       0.824776       0.777436       
328            0.965283       0.848825       0.700121       0.315438       
329            0.955183       0.593518       0.916906       0.200768       
330            1.004781       0.642493       0.912161       0.768178       
331            0.000000       0.558674       0.000000       0.511486       
332            0.731481       0.492379       0.463930       0.449792       
333            0.854044       0.753397       0.723587       0.808866       
334            0.866464       0.269304       0.810581       0.106199       
335            0.986938       1.097916       0.919224       0.985971       
336            0.000000       0.781081       0.000000       0.331923       
337            0.980525       0.576701       0.830921       0.520594       
338         

433            0.000000       0.851537       0.000000       0.435972       
434            0.724388       0.784046       0.625133       0.856095       
435            0.977644       1.004573       0.858785       0.870554       
436            0.866114       1.035380       0.720548       0.962705       
437            0.984688       0.500071       0.735188       0.963234       
438            1.085889       0.300102       0.951155       0.168609       
439            0.967270       1.009679       0.903103       0.438438       
440            0.000000       0.695631       0.000000       0.365514       
441            0.999464       0.709805       0.885474       0.597981       
442            0.000000       0.433657       0.000000       0.323230       
443            0.000000       0.286786       0.000000       0.411887       
444            0.908245       0.983799       0.779393       0.850024       
445            0.988537       0.558518       0.808351       0.280028       
446         

541            0.000000       0.302883       0.000000       0.265781       
542            0.925546       1.034479       0.879027       1.000000       
543            0.702743       0.629608       0.534612       0.396134       
544            0.988259       0.870269       0.785173       0.458394       
545            0.899698       0.324428       0.751493       0.767836       
546            0.851365       0.784726       0.581361       0.254226       
547            0.000000       0.748274       0.000000       0.629285       
548            0.000000       0.606238       0.000000       0.483660       
549            0.786084       0.770731       0.520264       0.594018       
550            0.921688       0.145203       0.860529       0.222920       
551            0.961010       0.392484       0.874463       0.271969       
552            0.000000       0.362840       0.000000       0.268928       
553            0.980912       1.179011       0.919919       1.000000       
554         

650            0.831857       0.674083       0.816506       0.528915       
651            1.088927       0.873809       0.910044       0.766489       
652            0.914973       0.477411       0.703902       0.263966       
653            0.899358       1.222781       0.832540       0.988176       
654            0.000000       0.449180       0.000000       0.233785       
655            0.904636       0.242833       0.753865       0.290755       
656            0.809918       0.999783       0.670819       0.758441       
657            0.000000       0.761860       0.000000       0.323311       
658            1.164472       0.579343       0.967860       0.804851       
659            1.006480       0.899760       0.922512       0.614206       
660            0.879905       1.066742       0.856636       0.981525       
661            0.961418       0.996293       0.867645       0.752576       
662            0.000000       0.768209       0.000000       0.264508       
663         

761            0.000000       0.908110       0.000000       0.466993       
762            0.937419       1.122490       0.875558       0.981575       
763            0.957628       1.040002       0.863823       0.972695       
764            0.992754       0.544504       0.693148       0.627130       
765            0.867684       0.264101       0.666814       0.007630       
766            0.929876       1.170579       0.899808       0.994573       
767            0.000000       0.406094       0.000000       0.147428       
768            0.835349       1.161269       0.739124       1.000000       
769            0.967978       0.858894       0.887282       0.891737       
770            0.000000       0.275587       0.000000       0.198547       
771            0.000000       0.425885       0.000000       0.302941       
772            0.803976       0.743295       0.616315       0.825987       
773            0.000000       0.146549       0.000000       0.128443       
774         

869            0.835298       0.166146       0.733080       0.084080       
870            0.932124       0.099584       0.701172       0.119094       
871            1.054901       0.797569       0.894728       0.555417       
872            0.986498       0.351669       0.855085       0.194666       
873            0.919181       0.773448       0.500891       0.569208       
874            0.000000       0.453545       0.000000       0.287477       
875            1.126267       0.808459       0.969648       0.596326       
876            0.000000       0.280857       0.000000       0.344275       
877            0.764591       0.172320       0.543188       0.012190       
878            0.868591       0.845321       0.769887       0.722609       
879            0.701052       0.829476       0.487589       0.571083       
880            0.786213       1.032790       0.613881       0.598720       
881            1.004601       0.438064       0.890828       0.116813       
882         

In [11]:
d1 = np.array(dist_ans)
d2 = np.array(dist_que)
d3 = np.array(tdist_ans)
d4 = np.array(tdist_que)
d5 = np.array(df[2])

d1.resize((931,1))
d2.resize((931,1))
d3.resize((931,1))
d4.resize((931,1))
d5.resize((931,1))

In [12]:
wmdata = pd.DataFrame(np.hstack((d2, d1,d4, d3, d5)))

In [13]:
wmdata

,0,1,2,3,4
0,0.180997,0.000000,0.020760,0.000000,1.0
1,0.790473,0.939044,0.200320,0.803154,0.0
2,0.756303,0.929330,0.745459,0.813780,0.0
3,1.220441,1.086222,1.000000,0.926564,0.0
4,0.801635,1.015583,0.741938,0.912896,0.0
5,0.463670,0.000000,0.458869,0.000000,1.0
6,1.290149,0.979779,1.000000,0.849560,0.0
7,0.492047,0.000000,0.217224,0.000000,1.0
8,0.000000,1.114567,0.103848,0.975416,0.0
9,0.773614,0.000000,0.273015,0.000000,0.0


In [14]:
wmdata.shape

(931, 5)

In [15]:
wmdata.to_csv('wmd_distance_ans.csv', header=None, index=None)

In [16]:
dist = pd.read_csv('wmd_distance_ans.csv', header=None, index_col=None)

In [17]:
dist.head()

,0,1,2,3,4
0,0.180997,0.000000,0.020760,0.000000,1.0
1,0.790473,0.939044,0.200320,0.803154,0.0
2,0.756303,0.929330,0.745459,0.813780,0.0
3,1.220441,1.086222,1.000000,0.926564,0.0
4,0.801635,1.015583,0.741938,0.912896,0.0


In [18]:

train_data, test_data = train_test_split(dist, test_size = 0.1, random_state = 0)

In [19]:
X_train, Y_train = train_data.loc[:, :3], train_data.loc[:, 4]

In [20]:
X_test, Y_test = test_data.loc[:, :3], test_data.loc[:, 4]

In [21]:
xgb_model = xgb.XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, objective='binary:logistic', eta=0.3, silent=1, subsample=0.8).fit(X_train, Y_train) 
xgb_prediction = xgb_model.predict(X_test)


In [22]:
print(accuracy_score(xgb_prediction, Y_test))

0.8404255319148937


In [23]:
print(X_test.head())

            0         1         2         3
386  0.303040  1.004868  0.232311  0.949484
496  0.822017  1.040137  0.620424  0.858445
878  0.845321  0.868591  0.722609  0.769887
910  1.167967  0.952650  0.994048  0.840862
478  0.723333  0.812576  0.528186  0.795176


In [24]:
cm = confusion_matrix(xgb_prediction , Y_test)
print(cm)

[[52  8]
 [ 7 27]]


In [25]:
f1 = f1_score(xgb_prediction , Y_test, average='macro')
f1

0.8282791377420534

In [26]:
print(classification_report(xgb_prediction , Y_test))

              precision    recall  f1-score   support

         0.0       0.88      0.87      0.87        60
         1.0       0.77      0.79      0.78        34

   micro avg       0.84      0.84      0.84        94
   macro avg       0.83      0.83      0.83        94
weighted avg       0.84      0.84      0.84        94



In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [28]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
print(acc_log)
acc_log = round(logreg.score(X_test, Y_test) * 100, 2)
print(acc_log)

f1 = f1_score(Y_pred , Y_test, average='macro')
print(f1)

84.95
81.91
0.8053830227743272


/home/stud/btech/cse/2017/rahul.cs17/anaconda3/envs/rahul/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [29]:
random_forest = RandomForestClassifier(n_estimators=5)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
print(acc_random_forest)
acc_random_forest = round(random_forest.score(X_test, Y_test) * 100, 2)
print(acc_random_forest)
f1 = f1_score(Y_pred , Y_test, average='macro')
print(f1)

96.77
78.72
0.7771455666192507


In [30]:
linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
print(acc_linear_svc)
acc_linear_svc = round(linear_svc.score(X_test, Y_test) * 100, 2)
print(acc_linear_svc)
f1 = f1_score(Y_pred , Y_test, average='macro')
print(f1)

85.19
82.98
0.8179176755447942


In [31]:
sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
print(acc_sgd)
acc_sgd = round( sgd.score(X_test, Y_test) * 100, 2)
print(acc_sgd)
f1 = f1_score(Y_pred , Y_test, average='macro')
print(f1)

84.71
85.11
0.8365623447590662


/home/stud/btech/cse/2017/rahul.cs17/anaconda3/envs/rahul/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [32]:
gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)

acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
print(acc_gaussian)
acc_gaussian = round(gaussian.score(X_test, Y_test) * 100, 2)
print(acc_gaussian)
Y_pred = gaussian.predict(X_test)
f1 = f1_score(Y_pred , Y_test, average='macro')
print(f1)


85.42
81.91
0.8076321174912724


In [33]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
print(acc_knn)
acc_knn = round(knn.score(X_test, Y_test) * 100, 2)
print(acc_knn)

Y_pred = knn.predict(X_test)
f1 = f1_score(Y_pred , Y_test, average='macro')
print(f1)

89.01
84.04
0.8350298350298351


In [34]:
def predict(q1, q2, ans1, ans2):
    q1 = clean(q1)
    q2 = clean(q2)
    ans1 = clean(ans1)
    ans2 = clean(ans2)
    
    da = distance(ans1, ans2)
    dq = distance(q1, q2)

    
    t1 = tfidf_vect.transform([ans1])
    t2 = tfidf_vect.transform([ans2])
    tda = cosine_similarity(t1, t2)

    
    t1 = tfidf_vect.transform([q1])
    t2 = tfidf_vect.transform([q2])
    tdq = cosine_similarity(t1, t2)

    X = np.zeros((1,4))
    
    X[0][0] = dq
    X[0][1] = da
    X[0][2] = tdq
    X[0][3] = tda

#     X[0] = dq
#     X[1] = da
#     X[2] = tdq
#     X[3] = tda
#     X.reshape((1,4))
    print(X)
    #print(xgb_model.predict(X))
    print(logreg .predict(X))
    print(random_forest.predict(X))
    print(linear_svc.predict(X))
    print(sgd.predict(X))
    print(gaussian.predict(X))
    print(knn.predict(X))
    print(votingC.predict(X))
    

In [36]:
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

In [37]:
votingC = VotingClassifier(estimators=[('lg', logreg), ('rf', random_forest),
 ('svc' , linear_svc),('sgd',sgd),('gbc',gaussian), ('knn', knn)])
votingC = votingC.fit(X_train, Y_train)

/home/stud/btech/cse/2017/rahul.cs17/anaconda3/envs/rahul/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/stud/btech/cse/2017/rahul.cs17/anaconda3/envs/rahul/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [38]:
q1 = "what is placement scenerio at IIT Delhi"
q2 = "what is placement scenerio at IIT Patna"

ans1 = "Depends on branch.. cs has the highest placements at iitp. Most of the companies visiting campus are from cs/it industry. PSU like coal india, isro, and some private industries visit for Mech. You can see details on iitp's website."
ans2  = "I don’t have a more recent report. But I guess this one here sums up things pretty well. Placements 2015: A Lookback"
predict(q1, q2, ans1, ans2)

[[0.14299542 1.1354653  0.33485552 0.94315148]]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]


In [39]:
Y_pred = votingC.predict(X_test)

In [40]:
print(votingC.score(X_train, Y_train))

0.8566308243727598


In [41]:
print(votingC.score(X_test, Y_test))

0.8191489361702128


In [42]:
print(f1_score(Y_pred, Y_test))

0.7605633802816902
